In [1]:
import csv
import re
import codecs

import numpy as np
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras import losses

/home/trip-3r/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def readData(path):
    data = []
    with open(path) as file:
        data = file.read()
#         data = codecs.decode(data, 'unicode_escape')
        data = data.split('\n')[:-1]
    return data

def removePattern(tweet, pattern):
    r = re.findall(pattern, tweet)
    for i in r:
        tweet = re.sub(i, '', tweet)
    return tweet

def preprocess(data):
    cleanData = []
    for line in data:
        tId, tAt, tSent, tweet = line.split("\t")[:4] # Splitting by tabspace
        tweet = removePattern(tweet, "@[\w]*") # Removing @user tags
        tweet = tweet.replace("#", "") # Removing '#' from hashtags
        tweet = tweet.replace("[^a-zA-Z#]", " ") # Removing punctuation and special characters
        tweet = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',"<URL>", tweet)
        tweet = re.sub(" +", " ", tweet)
        tweet = tweet.lower()
        cleanData.append([tId, tAt, tSent, tweet])
    return cleanData

def tokenize(tweet):
    return TweetTokenizer().tokenize(tweet)

In [3]:
def evaluate(target, predicted):
    f1 = f1_score(target, predicted, average='weighted')
    acc = accuracy_score(target, predicted)
    rec = recall_score(target, predicted, average = 'macro')
    print("F1 score : ", f1)
    print("Avg Recall:", rec)    
    print("Accuracy:  ", acc)    

# 5 Class with SVM

In [4]:
TRAIN_DATA = "../data/Subtasks_CE/twitter-2016train-CE.txt"
TEST_DATA = "../data/Subtasks_CE/twitter-2016test-CE.txt"
DEV_DATA = "../data/Subtasks_CE/twitter-2016dev-CE.txt"
DEVTEST_DATA = "../data/Subtasks_CE/twitter-2016devtest-CE.txt"

In [5]:
en_stopwords = set(stopwords.words("english")) 
vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    ngram_range=(1, 1),
    stop_words = en_stopwords)

In [6]:
trainData   = readData(TRAIN_DATA)
testData    = readData(TEST_DATA)
devData     = readData(DEV_DATA)
devTestData = readData(DEVTEST_DATA)

In [7]:
trainData = trainData + devData + devTestData

In [8]:
trainData   = preprocess(trainData)
testData    = preprocess(testData)
devData     = preprocess(devData)
devTestData = preprocess(devTestData)

In [9]:
trainTweets = [x[3] for x in trainData]
X_train = np.array(trainTweets)
trainSents = [x[2] for x in trainData]
y_train = np.array(trainSents)

testTweets = [x[3] for x in testData]
X_test = np.array(testTweets)
y_test = [x[2] for x in testData]

In [10]:
X = np.append(X_train, X_test)
X = vectorizer.fit_transform(X)
n = X_train.shape[0]
X_train = X[:n]
X_test = X[n:]

In [ ]:
params = {'kernel':['linear', 'rbf'],'C':[0.001, 0.01, 0.03, 0.05, 0.07 ,0.1,1, 10],'gamma':[0.000000001,0.00001, 0.001,0.1]}
clf = GridSearchCV(SVC(), params, cv = 2, n_jobs = -1)
clf.fit(X_train, y_train)
print(clf.best_params_)
print(clf.score(X_test, y_test))

In [ ]:
evaluate(y_test, clf.predict(X_test))

# 5 Class with LSTM

In [ ]:
TRAIN_DATA = "../data/Subtasks_CE/twitter-2016train-CE.txt"
TEST_DATA = "../data/Subtasks_CE/twitter-2016test-CE.txt"
DEV_DATA = "../data/Subtasks_CE/twitter-2016dev-CE.txt"
DEVTEST_DATA = "../data/Subtasks_CE/twitter-2016devtest-CE.txt"

In [ ]:
trainData   = readData(TRAIN_DATA)
testData    = readData(TEST_DATA)
devData     = readData(DEV_DATA)
devTestData = readData(DEVTEST_DATA)

In [ ]:
trainData   = preprocess(trainData)
testData    = preprocess(testData)
devData     = preprocess(devData)
devTestData = preprocess(devTestData)

In [ ]:
trainData = trainData + devData + devTestData

In [ ]:
trainTweets = [x[3] for x in trainData]
X_train = np.array(trainTweets)
trainSents = [x[2] for x in trainData]
y_train = np.array(trainSents)

testTweets = [x[3] for x in testData]
X_test = np.array(testTweets)
testSents = [x[2] for x in testData]
y_test = np.array(testSents)

In [ ]:
max_features = 2000
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(trainTweets)
n = len(X_train)
X = tokenizer.texts_to_sequences(trainTweets + testTweets)
X_train = X[:n]
X_test = X[n:]
X_train = pad_sequences(X_train, maxlen=32)
X_test = pad_sequences(X_test, maxlen=32)

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
Y_train = []
Y_test = []
for i in trainSents:
    if i == "-2":
        Y_train.append([1, 0, 0, 0, 0])
    elif i == "-1":
        Y_train.append([0, 1, 0, 0, 0])
    elif i == "0":
        Y_train.append([0, 0, 1, 0, 0])
    elif i == "1":
        Y_train.append([0, 0, 0, 1, 0])
    elif i == "2":
        Y_train.append([0, 0, 0, 0, 1])
    else:
        print(i)
        
Y_train = np.array(Y_train)

for i in testSents:
    if i == "-2":
        Y_test.append([1, 0, 0, 0, 0])
    elif i == "-1":
        Y_test.append([0, 1, 0, 0, 0])
    elif i == "0":
        Y_test.append([0, 0, 1, 0, 0])
    elif i == "1":
        Y_test.append([0, 0, 0, 1, 0])
    elif i == "2":
        Y_test.append([0, 0, 0, 0, 1])
    else:
        print(i)
Y_test = np.array(Y_test)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 7, batch_size=batch_size, verbose = 2)

In [ ]:
preds = []

for x in range(len(X_test)):
    result = model.predict(X_test[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == 0:
        preds.append([1, 0, 0, 0, 0])
    elif np.argmax(result) == 1:
        preds.append([0, 1, 0, 0, 0])
    elif np.argmax(result) == 2:
        preds.append([0, 0, 1, 0, 0])
    elif np.argmax(result) == 3:
        preds.append([0, 0, 0, 1, 0])
    elif np.argmax(result) == 4:
        preds.append([0, 0, 0, 0, 1])
        
preds = np.array(preds)

In [ ]:
evaluate(preds, Y_test)


# 2 class LSTM

In [ ]:
TRAIN_DATA = "../data/Subtasks_BD/twitter-2016train-BD.txt"
TEST_DATA = "../data/Subtasks_BD/twitter-2016test-BD.txt"
DEV_DATA = "../data/Subtasks_BD/twitter-2016dev-BD.txt"
DEVTEST_DATA = "../data/Subtasks_BD/twitter-2016devtest-BD.txt"

In [ ]:
trainData   = readData(TRAIN_DATA)
testData    = readData(TEST_DATA)
devData     = readData(DEV_DATA)
devTestData = readData(DEVTEST_DATA)

In [ ]:
trainData   = preprocess(trainData)
testData    = preprocess(testData)
devData     = preprocess(devData)
devTestData = preprocess(devTestData)

In [ ]:
trainData = trainData + devData + devTestData

In [ ]:
trainTweets = [x[3] for x in trainData]
X_train = np.array(trainTweets)
trainSents = [x[2] for x in trainData]
y_train = np.array(trainSents)

testTweets = [x[3] for x in testData]
X_test = np.array(testTweets)
testSents = [x[2] for x in testData]
y_test = np.array(testSents)

In [ ]:
max_features = 2000
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(trainTweets)
n = len(X_train)
X = tokenizer.texts_to_sequences(trainTweets + testTweets)
X_train = X[:n]
X_test = X[n:]
X_train = pad_sequences(X_train, maxlen=32)
X_test = pad_sequences(X_test, maxlen=32)

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
Y_train = []
Y_test = []
for i in trainSents:
    if i == "positive":
        Y_train.append([1, 0])
    else:
        Y_train.append([0, 1])
Y_train = np.array(Y_train)

for i in testSents:
    if i == "positive":
        Y_test.append([1, 0])
    else:
        Y_test.append([0, 1])
Y_test = np.array(Y_test)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 7, batch_size=batch_size, verbose = 2)

In [ ]:
preds = []
for x in range(len(X_test)):
    result = model.predict(X_test[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == 0:
        preds.append([1, 0])
    else:
        preds.append([0, 1])
preds = np.array(preds)

In [ ]:
evaluate(preds, Y_test)